In [2]:
!pip install datasets keras-tuner contractions pyspellchecker langdetect deep-translator

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 26.6 MB/s eta 0:00:0000:01
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 88.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.7/110.7 kB 8.4 MB/s eta 0:00:00
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993227 sha256=0b9ffbbf318e1272a64e05ef578b0784a8a95c0369a221f681cf8558e430ee0d
  Stored in directory: /root/.cache/pip/wheels/95/03/7d/59ea870c70ce4e5a370638b5462a7711ab78fba2f655d05106
Successfully built langdetect


In [3]:
import numpy as np

import torch

import torch.nn as nn

from tensorflow.keras.models import Sequential

from tensorflow.keras.layers import Dense, Dropout, Embedding, Bidirectional, GRU, Layer, Input, Conv1D, MaxPooling1D, GlobalMaxPooling1D, LSTM, Concatenate, MultiHeadAttention, Add,LayerNormalization, BatchNormalization

from tensorflow.keras.preprocessing.text import Tokenizer

from keras.preprocessing.sequence import pad_sequences

from tensorflow.keras.optimizers import Adam

from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

from datasets import load_dataset

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report, accuracy_score

import re

import matplotlib.pyplot as plt

import tensorflow as tf

from tensorflow import keras

import keras_tuner as kt

from torch.optim import AdamW

from torch.optim.lr_scheduler import ReduceLROnPlateau

from tqdm import tqdm

from transformers import BertTokenizer, BertModel

from torch.utils.data import DataLoader

from deep_translator import MyMemoryTranslator

from langdetect import detect

import spacy

import contractions

import nltk

from spellchecker import SpellChecker

import random

from GPUtil import showUtilization as gpu_usage

from numba import cuda

from datasets import load_dataset

import optuna

from tensorflow.keras.models import Model

from tensorflow.keras.regularizers import l2

from sklearn.linear_model import LogisticRegression

# Set a seed value

seed_value = 42



random.seed(seed_value)

# For NumPy

np.random.seed(seed_value)



# For TensorFlow

tf.random.set_seed(seed_value)



# For PyTorch

torch.manual_seed(seed_value)

if torch.cuda.is_available():

    torch.cuda.manual_seed(seed_value)

    torch.cuda.manual_seed_all(seed_value)  # if using multi-GPU

    torch.backends.cudnn.deterministic = True

    torch.backends.cudnn.benchmark = False

# Transformer Model (BERT + BiLSTM)

In [52]:
def free_gpu_cache():
    print("Initial GPU Usage")
    gpu_usage()                             

    torch.cuda.empty_cache()

    cuda.select_device(0)
    cuda.close()
    cuda.select_device(0)

    print("GPU Usage after emptying the cache")
    gpu_usage()

free_gpu_cache()                           


  Preparing metadata (setup.py) ... done
  Created wheel for GPUtil: filename=GPUtil-1.4.0-py3-none-any.whl size=7394 sha256=fce45d05bd5ee4dce50e8a1cc5c8f5d3c62f238ee883bb19c7d29db78c3399fb
  Stored in directory: /root/.cache/pip/wheels/a9/8a/bd/81082387151853ab8b6b3ef33426e98f5cbfebc3c397a9d4d0
Successfully built GPUtil
Initial GPU Usage
| ID | GPU | MEM |
------------------
|  0 |  0% | 99% |
GPU Usage after emptying the cache
| ID | GPU | MEM |
------------------
|  0 | 85% |  2% |


In [7]:
# Load the Rotten Tomatoes dataset
dataset = load_dataset("rotten_tomatoes")

# Split the dataset into training, validation, and test sets
train_dataset = dataset['train']
validation_dataset = dataset['validation']
test_dataset = dataset['test']


README.md:   0%|          | 0.00/7.46k [00:00<?, ?B/s]

train.parquet:   0%|          | 0.00/699k [00:00<?, ?B/s]

validation.parquet:   0%|          | 0.00/90.0k [00:00<?, ?B/s]

test.parquet:   0%|          | 0.00/92.2k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/8530 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1066 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1066 [00:00<?, ? examples/s]

In [14]:
# Initialize BERT tokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

# Tokenize the dataset
def tokenize_function(example):
    return tokenizer(example['text'], padding="max_length", truncation=True, max_length=128)

# Apply the tokenizer on the dataset
train_dataset = train_dataset.map(tokenize_function, batched=True)
validation_dataset = validation_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

# Convert datasets to PyTorch format
train_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])
validation_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])
test_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])

# DataLoaders
batch_size = 128
train_loader = DataLoader(train_dataset, shuffle=True, batch_size=batch_size)
validation_loader = DataLoader(validation_dataset, batch_size=batch_size)
test_loader = DataLoader(test_dataset, batch_size=batch_size)


Map:   0%|          | 0/8530 [00:00<?, ? examples/s]

Map:   0%|          | 0/1066 [00:00<?, ? examples/s]

Map:   0%|          | 0/1066 [00:00<?, ? examples/s]

In [23]:
class BertBiLSTM(nn.Module):
    def __init__(self, bert_model_name="bert-base-uncased", lstm_hidden_size=128, lstm_layers = 2, num_classes=2, dropout_rate = 0.1):
        super(BertBiLSTM, self).__init__()
        self.bert = BertModel.from_pretrained(bert_model_name, gradient_checkpointing=True)
        self.lstm = nn.LSTM(input_size=self.bert.config.hidden_size,
                            hidden_size=lstm_hidden_size,
                            num_layers=lstm_layers,
                            bidirectional=True,
                            batch_first=True)
        self.dropout = nn.Dropout(dropout_rate)
        self.classifier = nn.Linear(lstm_hidden_size * 2, num_classes)  # BiLSTM is bidirectional

    def forward(self, input_ids, attention_mask):
        # Get the BERT embeddings
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        sequence_output = outputs.last_hidden_state

        # Pass BERT embeddings through the BiLSTM
        lstm_output, _ = self.lstm(sequence_output)

        # Take the output of the LSTM's last time step
        lstm_output = lstm_output[:, -1, :]

        lstm_output = self.dropout(lstm_output)
        
        # Classify using a linear layer
        logits = self.classifier(lstm_output)

        return logits


In [19]:
# Initialize device, model, criterion, optimizer, and scheduler
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = BertBiLSTM(num_classes=2).to(device)
criterion = nn.CrossEntropyLoss().to(device)
optimizer = AdamW(model.parameters(), lr=2e-5)
scheduler = ReduceLROnPlateau(optimizer, mode="min", factor=0.1, patience=2, verbose=True)

# Early stopping parameters
early_stopping_patience = 3  # stop after 3 epochs with no improvement
best_val_loss = float("inf")
early_stop_counter = 0

# Training function
def train(model, dataloader, optimizer, criterion):
    model.train()
    total_loss, total_correct = 0, 0
    for batch in tqdm(dataloader):
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["label"].to(device)

        optimizer.zero_grad()
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        total_correct += (outputs.argmax(dim=1) == labels).sum().item()

    avg_loss = total_loss / len(dataloader)
    accuracy = total_correct / len(dataloader.dataset)
    return avg_loss, accuracy

# Evaluation function
def evaluate(model, dataloader, criterion):
    model.eval()
    total_loss, total_correct = 0, 0
    with torch.no_grad():
        for batch in dataloader:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["label"].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            loss = criterion(outputs, labels)

            total_loss += loss.item()
            total_correct += (outputs.argmax(dim=1) == labels).sum().item()

    avg_loss = total_loss / len(dataloader)
    accuracy = total_correct / len(dataloader.dataset)
    return avg_loss, accuracy

# Training loop with early stopping and learning rate scheduler
epochs = 10
for epoch in range(epochs):
    print(f"Epoch {epoch+1}/{epochs}")
    train_loss, train_accuracy = train(model, train_loader, optimizer, criterion)
    val_loss, val_accuracy = evaluate(model, validation_loader, criterion)

    # Print training and validation statistics
    print(f"Training loss: {train_loss:.4f}, Training accuracy: {train_accuracy:.4f}")
    print(f"Validation loss: {val_loss:.4f}, Validation accuracy: {val_accuracy:.4f}")

    # Step with the scheduler
    scheduler.step(val_loss)

    # Early stopping check
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        early_stop_counter = 0
        # Optionally save the best model
        torch.save(model.state_dict(), "best_model.pth")
    else:
        early_stop_counter += 1
        if early_stop_counter >= early_stopping_patience:
            print("Early stopping triggered.")
            break

# Load the best model (if early stopping saved it)
model.load_state_dict(torch.load("best_model.pth"))

# Evaluate on the test set
test_loss, test_accuracy = evaluate(model, test_loader, criterion)
print(f"Test loss: {test_loss:.4f}, Test accuracy: {test_accuracy:.4f}")


Epoch 1/10


100%|██████████| 67/67 [02:02<00:00,  1.84s/it]


Training loss: 0.4912, Training accuracy: 0.8089
Validation loss: 0.3716, Validation accuracy: 0.8508
Epoch 2/10


100%|██████████| 67/67 [02:04<00:00,  1.85s/it]


Training loss: 0.2984, Training accuracy: 0.8971
Validation loss: 0.3802, Validation accuracy: 0.8583
Epoch 3/10


...

Training loss: 0.1054, Training accuracy: 0.9719
Validation loss: 0.4419, Validation accuracy: 0.8612
Epoch 6/10


100%|██████████| 67/67 [02:04<00:00,  1.85s/it]


Training loss: 0.0783, Training accuracy: 0.9815
Validation loss: 0.4713, Validation accuracy: 0.8612
Early stopping triggered.


/tmp/ipykernel_30/715145176.py:88: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("best_model.pth"))


Test loss: 0.3788, Test accuracy: 0.8462


# BERT + BiLSTM hyperparameter tuning

In [24]:
def objective(trial):
    # Hyperparameters to test
    lstm_hidden_size = trial.suggest_int("lstm_hidden_size", 64, 256, step=64)
    lstm_layers = trial.suggest_int("lstm_layers", 1, 3)
    dropout_rate = trial.suggest_float("dropout_rate", 0.1, 0.5, step=0.1)
    learning_rate = trial.suggest_loguniform("learning_rate", 1e-5, 1e-3)

    # Initialize model, optimizer, criterion, scheduler
    model = BertBiLSTM(num_classes=2, lstm_hidden_size=lstm_hidden_size, lstm_layers=lstm_layers, dropout_rate=dropout_rate).to(device)
    criterion = nn.CrossEntropyLoss().to(device)
    optimizer = AdamW(model.parameters(), lr=learning_rate)
    scheduler = ReduceLROnPlateau(optimizer, mode="min", factor=0.1, patience=2, verbose=True)

    # Early stopping parameters
    early_stopping_patience = 3
    best_val_loss = float("inf")
    early_stop_counter = 0

    # Training loop for a limited number of epochs for tuning
    for epoch in range(10):  
        train_loss, train_accuracy = train(model, train_loader, optimizer, criterion)
        val_loss, val_accuracy = evaluate(model, validation_loader, criterion)

        # Scheduler step and early stopping
        scheduler.step(val_loss)
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            early_stop_counter = 0
        else:
            early_stop_counter += 1
            if early_stop_counter >= early_stopping_patience:
                break

    return best_val_loss

# Run the hyperparameter tuning
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=20)  # Adjust `n_trials` based on how many configurations you want to test

# Print best trial results
print("Best trial:")
trial = study.best_trial
print(f"  Loss: {trial.value}")
print("  Params: ")
for key, value in trial.params.items():
    print(f"    {key}: {value}")

[I 2024-11-07 14:55:41,917] A new study created in memory with name: no-name-2a68ef6b-20b5-490e-996f-25afc7b639c3
/tmp/ipykernel_30/410620912.py:8: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform("learning_rate", 1e-5, 1e-3)
100%|██████████| 67/67 [02:03<00:00,  1.84s/it]
[I 2024-11-07 15:08:22,274] Trial 0 finished with value: 0.38136272960238987 and parameters: {'lstm_hidden_size': 64, 'lstm_layers': 3, 'dropout_rate': 0.5, 'learning_rate': 2.3155015612111282e-05}. Best is trial 0 with value: 0.38136272960238987.
100%|██████████| 67/67 [02:03<00:00,  1.85s/it]
[I 2024-11-07 15:18:58,703] Trial 1 finished with value: 0.69054204887814 and parameters: {'lstm_hidden_size': 128, 'lstm_layers': 1, 'dropout_rate': 0.5, 'learning_rate': 0.0004990775147916834}. Best is trial 0 with value: 0.38136

Best trial:
  Loss: 0.3159616043170293
  Params: 
    lstm_hidden_size: 256
    lstm_layers: 2
    dropout_rate: 0.2
    learning_rate: 0.00010609586394564347


In [33]:
best_params = trial.params
model = BertBiLSTM(
    num_classes=2,
    lstm_hidden_size=best_params["lstm_hidden_size"],
    lstm_layers=best_params["lstm_layers"],
    dropout_rate=best_params["dropout_rate"]
).to(device)
criterion = nn.CrossEntropyLoss().to(device)
optimizer = AdamW(model.parameters(), lr=best_params["learning_rate"])
scheduler = ReduceLROnPlateau(optimizer, mode="min", factor=0.1, patience=2, verbose=True)

# Early stopping parameters
early_stopping_patience = 2
best_val_loss = float("inf")
early_stop_counter = 0

epochs = 10
for epoch in range(epochs):
    train_loss, train_accuracy = train(model, train_loader, optimizer, criterion)
    val_loss, val_accuracy = evaluate(model, validation_loader, criterion)
    
    # Print training and validation statistics
    print(f"Training loss: {train_loss:.4f}, Training accuracy: {train_accuracy:.4f}")
    print(f"Validation loss: {val_loss:.4f}, Validation accuracy: {val_accuracy:.4f}")

    # Step with the scheduler
    scheduler.step(val_loss)

    # Early stopping check
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        early_stop_counter = 0
        torch.save(model.state_dict(), "best_model.pth")
    else:
        early_stop_counter += 1
        if early_stop_counter >= early_stopping_patience:
            print("Early stopping triggered.")
            break

# Load the best model and evaluate on the test set
model.load_state_dict(torch.load("best_model.pth"))
test_loss, test_accuracy = evaluate(model, test_loader, criterion)
print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")

100%|██████████| 67/67 [02:07<00:00,  1.90s/it]


Training loss: 0.4182, Training accuracy: 0.8123
Validation loss: 0.3287, Validation accuracy: 0.8565


100%|██████████| 67/67 [02:07<00:00,  1.91s/it]


Training loss: 0.1891, Training accuracy: 0.9306
Validation loss: 0.3900, Validation accuracy: 0.8574


100%|██████████| 67/67 [02:07<00:00,  1.91s/it]


Training loss: 0.0925, Training accuracy: 0.9706
Validation loss: 0.4485, Validation accuracy: 0.8583
Early stopping triggered.


/tmp/ipykernel_30/3549608279.py:41: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("best_model.pth"))


Test Loss: 0.3414, Test Accuracy: 0.8518
